In [1]:
import requests
import os
import shutil

In [17]:
# years = list(range(1991,2022))
years = list(range(1984,2023))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+", encoding= 'utf-8') as f:
        f.write(data.text)

In [18]:
from bs4 import BeautifulSoup
import pandas as pd
dfs = []
years = list(range(1984,2023))
for year in years:
    with open("mvp/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year
    dfs.append(mvp_df)

In [20]:
mvps = pd.concat(dfs)

mvps.tail()
mvps.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Larry Bird,27,BOS,52.0,652.0,760,0.858,79,38.3,...,10.1,6.6,1.8,0.9,0.492,0.247,0.888,13.6,0.215,1984
1,2,Bernard King,27,NYK,11.0,373.0,760,0.491,77,34.6,...,5.1,2.1,1.0,0.2,0.572,0.000,0.779,12.1,0.218,1984
2,3,Magic Johnson,24,LAL,5.0,305.0,760,0.401,67,38.3,...,7.3,13.1,2.2,0.7,0.565,0.207,0.810,10.2,0.191,1984
3,4,Kareem Abdul-Jabbar,36,LAL,3.0,153.0,760,0.201,80,32.8,...,7.3,2.6,0.7,1.8,0.578,0.000,0.723,8.9,0.163,1984
4,5,Isiah Thomas,22,DET,1.0,115.0,760,0.151,82,36.7,...,4.0,11.1,2.5,0.4,0.462,0.338,0.733,9.1,0.144,1984


In [21]:
mvps.to_csv("mvps.csv")

In [24]:

import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
years = list(range(1984,2023))
for year in years:
    url = player_stats_url.format(year)
    
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    with open("player/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(driver.page_source)

In [25]:
from bs4 import BeautifulSoup
import pandas as pd

dfs = []
years = list(range(1984,2023))
for year in years:
    with open("player/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

In [27]:
players = pd.concat(dfs)
players.head()
players.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
836,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
837,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
838,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
839,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022
840,605,Ivica Zubac,C,24,LAC,76,76,24.4,4.1,6.5,...,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,2022


In [28]:
players.to_csv("players.csv")

In [30]:
import requests
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
years = list(range(2021,2023))
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("team/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

In [31]:
dfs = []
years = list(range(1984,2023))
for year in years:
    with open("team/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

In [32]:
teams = pd.concat(dfs)

In [33]:
teams.head()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,62,20,.756,—,112.1,105.6,6.42,1984,Boston Celtics*
1,52,30,.634,10.0,107.8,105.6,2.39,1984,Philadelphia 76ers*
2,47,35,.573,15.0,106.9,103.0,3.79,1984,New York Knicks*
3,45,37,.549,17.0,110.0,108.9,1.27,1984,New Jersey Nets*
4,35,47,.427,27.0,102.7,105.6,-2.36,1984,Washington Bullets*


In [34]:
teams.to_csv("teams.csv")